In [1]:
import sys
sys.path.insert(0, "/export/servers/wenhao/database_reader/")

%load_ext autoreload
%autoreload 2

In [2]:
import database_reader as dr

/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework

In [3]:
import database_reader.utils as du

In [4]:
from database_reader import CPSC2018

In [5]:
hehe_cpsc = CPSC2018(db_path="/export/servers/data/CPSC2018/Training_WFDB/", verbose=5)

In [ ]:
ro = 608

In [ ]:
hehe_cpsc.load_ann(ro)

In [ ]:
hehe_cpsc.load_data(ro)[:,0]

In [ ]:
hehe_cpsc.load_ann(1)['df_leads']['lead_name'].tolist()

In [ ]:
for ro in range(1, hehe_cpsc.nb_records+1):
    try:
        ll = hehe_cpsc.load_ann(ro)['df_leads']['lead_name'].tolist()
        if ll != ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']:
            print(f"第{ro}个record的导联顺序不一样")
    except Exception as e:
        print(f"第{ro}个record发生了如下错误")
        print(e)

In [ ]:
for ro in range(1, hehe_cpsc.nb_records+1):
    try:
        ll = hehe_cpsc.load_data(ro)[:,0]
        fv = hehe_cpsc.load_ann(ro)['df_leads']['first_value'].values
        if (ll!=fv).any():
            print(f"第{ro}个record的导联顺序不一样")
    except Exception as e:
        print(f"第{ro}个record发生了如下错误")
        print(e)

In [ ]:
hehe_cpsc.get_labels(ro)

In [ ]:
hehe_cpsc.get_diagnosis(ro)

In [ ]:
hehe_cpsc.load_data(ro).shape

In [ ]:
hehe_cpsc.plot(ro)

In [15]:
hehe_cpsc.load_data(1).shape[0]//3000

2

In [10]:
input_len = 6*hehe_cpsc.freq

In [12]:
x,y = [], []

In [16]:
for ro in range(1, hehe_cpsc.nb_records+1):
    ecg = hehe_cpsc.load_data(ro)
    ann = hehe_cpsc.get_labels(ro)
    for i in range(ecg.shape[0]//input_len):
        x.append(ecg[input_len*i:input_len*(i+1)])
        y.append(ann)

In [25]:
((x[0]-np.mean(x[0]))/np.std(x[0])).shape

(3000, 12)

In [26]:
new_x = [(item-np.mean(item))/np.std(item) for item in x]

In [27]:
new_x = np.array(new_x)

In [28]:
new_x.shape

(14579, 3000, 12)

In [29]:
y = np.array(y)

In [32]:
np.save('./cpsc_y.npy', y)

In [33]:
np.save('./cpsc_x.npy', new_x)

In [ ]:
np.array([[1,1,10],[2,3,50]]) - np.array([])

In [45]:
hehe_cpsc.all_diagnosis

['N', 'AF', 'I-AVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']

In [47]:
new_y = np.zeros((new_x.shape[0],9))
new_y

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
for idx, item in enumerate(y):
    indices = [hehe_cpsc.all_diagnosis.index(l) for l in item]
    new_y[idx][indices] = 1

In [51]:
new_y[0],y[0], hehe_cpsc.all_diagnosis

(array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]),
 ['RBBB'],
 ['N', 'AF', 'I-AVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE'])

In [34]:
train_x = new_x[:12000]
test_x = new_x[12000:]

In [35]:
train_x.shape

(12000, 3000, 12)

In [52]:
train_y = new_y[:12000]
test_y = new_y[12000:]

In [53]:
from keras import layers
from keras import Input
from keras.models import Sequential, Model, load_model
from keras.layers import (
    LSTM, GRU,
    TimeDistributed, Bidirectional,
    ReLU, LeakyReLU,
    BatchNormalization,
    Dense, Dropout, Activation, Flatten, 
    Input, Reshape, GRU, CuDNNGRU,
    Conv1D,
    MaxPooling1D,
    GlobalAveragePooling1D, AveragePooling1D,
    concatenate,
)
from keras.initializers import he_normal, he_uniform, Orthogonal
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint


SEED = 42
freq = 500
cell_len_t = 6
nb_classes = 9
batch_size=128


TI_CNN = Sequential(name='TI_CNN')

TI_CNN.add(
    Conv1D(
        input_shape = (freq*cell_len_t, 12),
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)

TI_CNN.add(
    LSTM(
        128, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        32, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        9, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=False,
    )
)

TI_CNN.add(
    Dense(nb_classes,activation='sigmoid')
)


In [ ]:
TI_CNN.summary()

Model: "TI_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_27 (Conv1D)           (None, 3000, 64)          2368      
_________________________________________________________________
batch_normalization_27 (Batc (None, 3000, 64)          256       
_________________________________________________________________
re_lu_27 (ReLU)              (None, 3000, 64)          0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 3000, 64)          12352     
_________________________________________________________________
batch_normalization_28 (Batc (None, 3000, 64)          256       
_________________________________________________________________
re_lu_28 (ReLU)              (None, 3000, 64)          0         
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 1000, 64)          0    

In [ ]:

TI_CNN.compile(loss='binary_crossentropy', optimizer=Adam(0.0001))

checkpointer = ModelCheckpoint(filepath='./ckpt/weights.hdf5', verbose=1, monitor='val_acc', save_best_only=True)


In [ ]:


TI_CNN.fit(train_x, train_y, batch_size=128, epochs=200, verbose=1, validation_data=(test_x, test_y), callbacks=[checkpointer])

Instructions for updating:
Use tf.cast instead.
Train on 12000 samples, validate on 2579 samples
Epoch 1/200
12000/12000 [==============================] - 310s 26ms/step - loss: 0.5346 - val_loss: 0.4976

Epoch 00001: saving model to /tmp/weights.hdf5
Epoch 2/200
12000/12000 [==============================] - 205s 17ms/step - loss: 0.4627 - val_loss: 0.4511

Epoch 00002: saving model to /tmp/weights.hdf5
Epoch 3/200
12000/12000 [==============================] - 203s 17ms/step - loss: 0.4287 - val_loss: 0.4281

Epoch 00003: saving model to /tmp/weights.hdf5
Epoch 4/200
12000/12000 [==============================] - 206s 17ms/step - loss: 0.4039 - val_loss: 0.4149

Epoch 00004: saving model to /tmp/weights.hdf5
Epoch 5/200
12000/12000 [==============================] - 208s 17ms/step - loss: 0.3851 - val_loss: 0.3939

Epoch 00005: saving model to /tmp/weights.hdf5
Epoch 6/200
12000/12000 [==============================] - 205s 17ms/step - loss: 0.3688 - val_loss: 0.3813

Epoch 00006: s

12000/12000 [==============================] - 237s 20ms/step - loss: 0.1674 - val_loss: 0.2415

Epoch 00051: saving model to /tmp/weights.hdf5
Epoch 52/200
12000/12000 [==============================] - 217s 18ms/step - loss: 0.1633 - val_loss: 0.2480

Epoch 00052: saving model to /tmp/weights.hdf5
Epoch 53/200
12000/12000 [==============================] - 204s 17ms/step - loss: 0.1607 - val_loss: 0.2374

Epoch 00053: saving model to /tmp/weights.hdf5
Epoch 54/200
12000/12000 [==============================] - 221s 18ms/step - loss: 0.1579 - val_loss: 0.2465

Epoch 00054: saving model to /tmp/weights.hdf5
Epoch 55/200
12000/12000 [==============================] - 246s 20ms/step - loss: 0.1577 - val_loss: 0.2672

Epoch 00055: saving model to /tmp/weights.hdf5
Epoch 56/200
12000/12000 [==============================] - 256s 21ms/step - loss: 0.1609 - val_loss: 0.2696

Epoch 00056: saving model to /tmp/weights.hdf5
Epoch 57/200
12000/12000 [==============================] - 258s 21ms/s

# 测试ATI-CNN

In [ ]:
from keras import layers
from keras import Input
from keras.models import Sequential, Model
from keras.models import Sequential, load_model
from keras.layers import (
    LSTM, GRU,
    TimeDistributed, Bidirectional,
    ReLU, LeakyReLU,
    BatchNormalization,
    Dense, Dropout, Activation, Flatten, 
    Input, Reshape, GRU, CuDNNGRU,
    Conv1D,
    MaxPooling1D,
    GlobalAveragePooling1D, AveragePooling1D,
    concatenate,
)
from keras.initializers import he_normal, he_uniform, Orthogonal

In [ ]:
SEED = 42
freq = 500
cell_len_t = 6

TI_CNN = Sequential(name='TI_CNN')

TI_CNN.add(
    Conv1D(
        input_shape = (freq*cell_len_t, 12),
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=64, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=128, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=256, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)

TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    Conv1D(
        filters=512, kernel_size=3, strides=1, padding='same',
        kernel_initializer=he_normal(SEED),
        )
)
TI_CNN.add(
    BatchNormalization()
)
TI_CNN.add(
    ReLU()
)
TI_CNN.add(
    MaxPooling1D(
        pool_size=3, strides=3,
    )
)

TI_CNN.add(
    LSTM(
        128, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        32, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=True,
    )
)

TI_CNN.add(
    LSTM(
        9, kernel_initializer=Orthogonal(seed=SEED),
        return_sequences=False,
    )
)


In [ ]:
TI_CNN.summary()

# 测试0236号算法

In [ ]:
import glob
import random
import os
import argparse
import scipy.io as sio
from keras import backend as K
from sklearn.model_selection import train_test_split
import csv
import numpy
import numpy as np

import pandas as pd
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape, GRU, CuDNNGRU
from keras.layers import Convolution1D, MaxPool1D, GlobalAveragePooling1D,concatenate,AveragePooling1D
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import Model
from keras.utils import multi_gpu_model
from keras import initializers, regularizers, constraints
from keras.layers import Layer
import numpy as np
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import scipy.io as sio
from os import listdir

In [ ]:
np.load("./magicVector_test_val_strategy.npy")

In [ ]:
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
batch_size = 64
num_classes = 9
epochs = 1000000000000000000000000000000000
magicVector = np.load('magicVector_test_val_strategy.npy')
leadsLabel = np.asarray(['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6'])

In [ ]:
main_input = Input(shape=(72000,12), dtype='float32', name='main_input')
x = Convolution1D(12, 3, padding='same')(main_input)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 48, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
cnnout = Dropout(0.2)(x)
x = Bidirectional(CuDNNGRU(12, input_shape=(2250,12),return_sequences=True,return_state=False))(cnnout)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = AttentionWithContext()(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
main_output = Dense(num_classes,activation='sigmoid')(x)

In [ ]:
keras.__version__

In [ ]:
m = Model(inputs=main_input, outputs=main_output)

In [ ]:
m.summary()